In [1]:
import requests
import json
import time

In [2]:
with open('../data/proposals/aave/aave_snapshot_proposals_1_year.json', 'r', encoding='utf8') as file:
    data = json.load(file)

In [10]:
url = "https://hub.snapshot.org/graphql"

def retrieve_votes(proposal):
    body = f"""
    {{
        votes(
            first: 100,
            where: {{
                space_in: ["aave.eth"],
                proposal: "{proposal}",
                vp_gt: 1.0,
            }},
            orderBy: "vp",
            orderDirection: desc
        ) {{
            voter
            created
            choice
            vp
        }}
    }}
    """
    
    response = requests.post(url=url, json={"query": body})
    
    if response.status_code == 200:
        return response.json().get("data", {}).get("votes", [])
    else:
        return []

In [ ]:
all_votes = []

for proposal in data['data']['proposals']:
    proposal_id = proposal['id']
    votes = retrieve_votes(proposal_id)
    
    all_votes.append({
        "proposal_id": proposal_id,
        "votes": votes
    })
    time.sleep(1)
    
with open ('../data/proposals/aave/votes.json', 'w', encoding='utf8') as outfile:
    json.dump(all_votes, outfile, ensure_ascii=False)